#### The following is the code for the parser I built from scratch. The outputs refer to the second dataset I grabbed. The link that I parse from is constantly updating so the movies in the range 1-140 would be different than the time I executed this code. 

After the first dataset I noticed I need more rotten reviews so this link is also filtered to show predominantly rotten movies

In [1]:
# importing all neccesarry libraries 
from selenium import webdriver
# service is an object that manages the starting and stopping of the ChromeDriver
from selenium.webdriver.chrome.service import Service
# some packages have their own types of errors -- we will use the No Such Element in our code later
from selenium.common.exceptions import NoSuchElementException
# the By class is used to locate elements within a document
from selenium.webdriver.common.by import By

# and some other packages 
import time 
import re 
from bs4 import BeautifulSoup

In [104]:

# initialize our browser
browser = webdriver.Chrome()

time.sleep(1)
browser.maximize_window()
time.sleep(1)

# putting in rotten tomatoes link 
link = "https://www.rottentomatoes.com/browse/movies_at_home/critics:rotten?page=5"
browser.get(link)

time.sleep(2)

#empty lists to store the data 
    
ratings = []
texts = []

for movie in range(1,140):

    link = "https://www.rottentomatoes.com/browse/movies_at_home/critics:rotten?page=5"
    browser.get(link)

    try:
        # try First XPath
        xp2 = (f"//*[@id='main-page-content']/div[1]/div[3]/div/div/div[{movie}]/div/tile-dynamic/a/span[1]")
        browser.find_element(By.XPATH,xp2).click()
    except NoSuchElementException:
        try:
            # try Second XPath
            xp2_Alt = (f"//*[@id='main-page-content']/div[1]/div[3]/div/div/div[{movie}]/a/tile-dynamic/div/span[1]")
            browser.find_element(By.XPATH,xp2_Alt).click()
        except NoSuchElementException:
            print(f"Movie element for index {movie} not found with either XPath.")
            continue

    # click on the all critics button

    try:
        # check if there is critic
        xp4 = "//*[@id='critics-reviews']/div/div/div[1]/a[1]"
        browser.find_element(By.XPATH,xp4).click()
        time.sleep(2)
    except NoSuchElementException:
        print(f'No critics or reviews found for movie at index {movie}')
        continue


    # grabbing all the elements of the reviews we want
    for i in range (1,2):
         num_of_clicks = 5
         # clicking on the load more button 
         for i in range (0,num_of_clicks):
            next_page_xpath = "//*[@id='reviews']/div[2]/rt-button"

            try:
                next_page_button = browser.find_element(By.XPATH, next_page_xpath)
                if next_page_button.is_displayed() and next_page_button.is_enabled():
                    next_page_button.click()
                    time.sleep(2)
                else:
                    print('Reached the last page or next page button is not interactable.')
            except NoSuchElementException:
                print('There is no next page button.')
            except ElementNotInteractableException:
                print('Next page button is not interactable.')
            continue
    
    # parse page to a soup
    page_source = browser.page_source
    soup = BeautifulSoup(page_source, 'html.parser')
    reviews_content = soup.find_all('div', class_ = 'review-row')

    # find locations of each element you want
    for review in reviews_content:
        rating = None
        text = None

        r = review.find('score-icon-critic-deprecated')
        t = review.find('p', class_ = 'review-text')
  
        # if they are found get the specific part of the element 
        if r:

            rating = r.attrs.get('state')
            text = t.get_text(strip = True)

        # add all of them into our empty lists
            ratings.append(rating)
            texts.append(text)

    

browser.quit()

Reached the last page or next page button is not interactable.
Reached the last page or next page button is not interactable.
Reached the last page or next page button is not interactable.
Reached the last page or next page button is not interactable.
Reached the last page or next page button is not interactable.
Reached the last page or next page button is not interactable.
Reached the last page or next page button is not interactable.
Reached the last page or next page button is not interactable.
Reached the last page or next page button is not interactable.
Reached the last page or next page button is not interactable.
Reached the last page or next page button is not interactable.
Reached the last page or next page button is not interactable.
Reached the last page or next page button is not interactable.
Reached the last page or next page button is not interactable.
Reached the last page or next page button is not interactable.
Reached the last page or next page button is not intera

In [105]:
# amount of reviews we parsed
print(len(texts))

12064


In [109]:
import csv

# create empty list for reviews
reviews_list = []

for n in range(len(ratings)):
    # clean '\n'
    cleaned_texts = texts[n].replace('\n', ' ')
    my_dict = {
        'rating': ratings[n],
        'text': cleaned_texts
    }
    reviews_list.append(my_dict)

# export to CSV
filename = 'Movie_Dataset.csv'
fieldnames = ['rating', 'text']

with open(filename, 'w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(reviews_list)



In [107]:
# counting fresh and rotten reviews 
count_r = 0
count_f = 0
for rating in ratings:
    if rating == 'rotten':
        count_r += 1
    elif rating == 'fresh':
        count_f +=1

print (f'There are {count_r} rotten reviews')
print (f'There are {count_f} fresh reviews')


There are 7414 rotten reviews
There are 4650 fresh reviews
